# **Pyodide**

**At the moment this python console CANNOT load due to incompatibility with the plotly sphinx extension**

You can execute any Python code. Just enter something in the box below and click the button.
Loaded packages by default are:
- `numpy` as `np`
- `matplotlib.pyplot` as `plt`
- `scipy` as `sp`

In [1]:
%%HTML
<!doctype html>
<html>
  <head>
    <script src="https://cdn.jsdelivr.net/pyodide/v0.25.0/full/pyodide.js"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/codemirror/5.63.1/codemirror.min.css">
    <script src="https://cdnjs.cloudflare.com/ajax/libs/codemirror/5.63.1/codemirror.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/codemirror/5.63.1/mode/python/python.min.js"></script>
	
    <style>
	.python {
		font-family: 'Cascadia Code', monospaced;
		font-size: 12px;
    }
    .CodeMirror {
		height: auto;
		border: 1px solid #ddd;
    }
    .CodeMirror-scroll {
		max-height: 300px;
    }
    #output {
		width: 100%;
		height: 200px;
		background-color: white;
		color: black;
		font-family: 'Cascadia Code', monospace;
		font-size: 14px;
		border: 1px solid #ddd;
		overflow-y: auto;
    }	  
	#run-button {
		background-color: #4CAF50; /* Green */
		border: none;
		color: white;
		padding: 10px 20px;
		text-align: center;
		text-decoration: none;
		display: inline-block;
		font-size: 16px;
		margin: 4px 2px;
		cursor: pointer;
		border-radius: 5px;
    }

    #run-button::before {
		content: '\25B6'; /* Unicode character for a right-pointing triangle */
		margin-right: 8px;
    }
	
	#figs {
		overflow: hidden;
		transition: height 0.3s ease;
		height: 30px; /* Set the initial height */
		border: 1px solid #ccc;
		padding: 10px;
    }
    </style>    
  </head>

  <body>
    <textarea id="code" class="python" rows = 30 style="width: 100%;">
# Author: Elia Ferretti
# November 2023

# Let's start simple
print("Hello world")
dp = 0

# try a for loop
a = 0
for i in range(0,1001):
    a += i
print("The sum of the integers from 1 to 1000 is:\t",a)

# Let's try to plot
x = np.linspace(0,10,100)

plt.figure(1)
plt.title("sine and cosine")
plt.plot(x,np.sin(x),label="sine")
plt.plot(x,np.cos(x),label="cosine")
plt.legend(); plt.grid(); plt.show()    
    </textarea>

    <script>
        var editor = CodeMirror.fromTextArea(document.getElementById("code"), {
            mode: "python",
            theme: "default", // You can choose a different theme if needed
            lineNumbers: true,
            autofocus: true
        });
        editor.on("change", function() {
            // This function will be called whenever the content of the editor changes
            console.log("Editor content changed:", editor.getValue());
        });
		document.addEventListener("keydown", function (event) {
			if (event.ctrlKey && event.key === "Enter") {
				evaluatePython();
				event.preventDefault(); // Prevent default Ctrl+Enter behavior (e.g., new line in textarea)
			}
		});
    </script>
	
	<br />
    <button id="run-button" onclick="evaluatePython()">Run</button>
    
	<br />
    <br />
    <textarea id="output" style="width: 100%;" rows="6" disabled></textarea>

	<div id="figs " style="width: 100%;" disabled></div>

	
    <script>
		const output = document.getElementById("output");
		const code = document.getElementById("code");

		function addToOutput(s) {
			if (output.value == "Ready!\n") {
				output.value = s
			} else {
				output.value += s;
			}
			output.scrollTop = output.scrollHeight;  // Auto-scroll to the bottom
		}

		output.value = "Initializing...\n";
		// init Pyodide
		async function main() {
		let pyodide = await loadPyodide();
		// Load additional python packages using loadPackage
		await pyodide.loadPackage(['numpy', 'matplotlib', 'scipy']);
		output.value = "Ready!\n";
		return pyodide;
		}
		let pyodideReadyPromise = main();

		async function evaluatePython() {
			let pyodide = await pyodideReadyPromise;
			try {
				
				code.value = editor.getValue();							// get the code from the editor
				code.value = code.value.replace('plt.show();','')			// remove plt.show(); as it adds a figure space
				code.value = code.value.replace('plt.show()','')			// at the bottom of the page

				// Redirect sys.stdout to capture output
				let capturedOutput = '';
				pyodide.runPython(`
				import sys
				from io import StringIO
				sys.stdout = StringIO()
				sys.stderr = StringIO()
				`);

				
				pyodide.runPython('import time');
				pyodide.runPython('___start_time = time.time()');	
				// Load necessary/main libraries
				pyodide.runPython('import numpy as np');
				pyodide.runPython('import matplotlib.pyplot as plt');
				pyodide.runPython('import scipy as sp');

				// clear lines from plot	
				pyodide.runPython('plt.clf()');
				pyodide.runPython(code.value); //this line is necessary to capture the output	
				// Capture the figure as an image
				let captureFigure = pyodide.runPython('plt.gcf()');
				let imageStream = pyodide.runPython(`
				  from io import BytesIO
				  import base64
				  image_stream = BytesIO()
				  plt.savefig(image_stream, format='png')
				  image_stream.seek(0)
				  base64.b64encode(image_stream.read()).decode('utf-8')
				`);

				// Update the content of the HTML element with id "figs "
				let figElement = document.getElementById('figs ');
				figElement.innerHTML = `<img src="data:image/png;base64,${imageStream}" style="width:100%;" alt="Python Figure">`;
				
				//print execution time
				pyodide.runPython('print("______________________________________")');
				pyodide.runPython('print("Executed in "+str(time.time()-___start_time)+" seconds")');
				// Get captured output
				capturedOutput = pyodide.runPython('sys.stdout.getvalue()'); 
				//let variableValue = pyodide.globals.get("dp");  
				let capturedError = pyodide.runPython('sys.stderr.getvalue()');

				if (capturedError) {
				  addToOutput(capturedError);
				}
				//printing statements
				//addToOutput("dp = "+variableValue+"\n");
				
				addToOutput(capturedOutput+"\n");  
				} catch (err) {
				addToOutput(err);
			}
		}
    </script>
  </body>
</html>